# Machine Learning. Hyper Parameters Optimizaion

## Task Description

Data for this task has been colleted experimentally and it represents a biological process. It contains the following columns:
* Activity: actual biological response [0, 1]
* D1-D1776: molecular descriptors like size, form and chemical elements.

Data preprocessing is not required, data is already encoded and normalized.

F1-score metric must be used within the task.

Two models must be trained: logistic regression and random forest. Further a hyper parameter optimization must be performed using basic and advanced methods (GridSearchCV, RandomizedSearchCV, Hyperopt, Optuna). Maximum number of iterations must not exceed 50.

## Initialization

### Import Necessary Libraries

In [3]:
import optuna
import pandas as pd

from sklearn import model_selection

### Define Constants

In [ ]:
random_state = 42

## Load Data

In [2]:
data = pd.read_csv('../../data/_train_sem09.csv')
data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


### Split the data into train and test datasets

In [ ]:
X = data.drop('Activity', axis=1)
y = data['Activity']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, )